# **Combined Class Logistic Regression**

In [153]:
# Import Statements

# Standard Stuff
import numpy as np
from numpy import ma # (Masked Array): Has NumPy Functions That Work With NaN Data
from numpy.linalg import pinv # Moore-Penrose Pseudoinverse
import math

# Sklearn Imports 
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from scipy.special import expit
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.optimize import fmin_bfgs, minimize_scalar # BFGS Algorithm: Extremely Common Optimization Algorithm

# Scipy Imports
from scipy.optimize import minimize_scalar

class MultiClassLogisticRegression:

    def __init__(self, eta, iterations = 20, C = 0.0001, solver = "quasi", penalty = "l2", line_iters = 10):
        """
        MultiClassLogisticRegression Constructor

        Note: Internally We Will Store The Weights As self.w_ To Keep With Sklearn Conventions
        
        Parameters:
        - eta: Learning Rate
        - iterations: Number Of Iterations (Default: 20)
        - C: Regularization Strength (Default: 0.0001)
        - solver: Optimization Algorithm: ('steepest', 'stochastic', 'newton', 'quasi') (Default: 'quasi')
        - penalty: Regularization Penalty Type: ('none', 'l1', 'l2', 'both') (Default: 'l2')
        - line_iters: Number Iterations For Line Search (If Applicable)
        """
        self.eta = eta
        self.iters = iterations
        self.C = C
        self.solver = solver
        self.penalty = penalty
        self.line_iters = line_iters
        self.classifiers_ = []

    # Return Trained Status (String)
    def __str__(self):
        if hasattr(self, 'w_'):
            return 'MultiClass Logistic Regression Object With Coefficients:\n' + str(self.w_)
        else:
            return 'Untrained MultiClass Logistic Regression Object'

    # ==============================
    # Convenience Methods (Private):
    # ==============================

    @staticmethod
    def _add_bias(X):

        # Add Bias Term
        return np.hstack((np.ones((X.shape[0], 1)), X))
    
    @staticmethod
    def _sigmoid(theta):

        # Increase Stability, Redefine Sigmoid Operation (1 / (1 + np.exp(-theta)))
        return expit(theta)
    
    # Defines Function With First Input To Be Optimized (Eta)
    @staticmethod
    def _objective_function_steepest(eta, X, y, w, grad, C):
        wnew = w - grad * eta
        g = expit(X @ wnew)

        # The Line Search Looks For Minimization, Take Negative Of Log Likelihood
        return -np.sum(ma.log(g[y == 1])) - ma.sum(np.log(1 - g[y == 0])) + C * sum(wnew**2)
    
    @staticmethod
    def _objective_function_bfgs(w, X, y, C):
        g = expit(X @ w)

        # Invert This Because SciPy Minimizes, But We Derived All Formulas For Maximizing
        return -np.sum(ma.log(g[y == 1])) - np.sum(ma.log(1 - g[y == 0])) + C * sum(w**2) 

    @staticmethod
    def _objective_gradient_bfgs(w, X, y, C):
        g = expit(X @ w)
        ydiff = y - g
        gradient = np.mean(X * ydiff[:, np.newaxis], axis = 0)
        gradient = gradient.reshape(w.shape)
        gradient[1:] += -2 * w[1:] * C
        return -gradient
    
    # ============================
    # Binary Probability Functions
    # ============================

    def _predict_proba_binary(self, X, w, add_bias = True):

        # Add Bias Term If Requested
        Xb = self._add_bias(X) if add_bias else X

        # Return Probability Of Class 1
        return self._sigmoid(Xb @ w)
    
    def _predict_binary(self, X):

        # Return Actual Prediction
        return (self._predict_proba_binary(X, self.w_) > 0.5)
    
    # ==================
    # Gradient Functions
    # ==================

    # Vectorized Gradient Calculation With Chosen Regularization
    def _get_gradient(self, X, y):

        # Get Difference, Make Column Vector And Multiply Through
        ydiff = y - self._predict_proba_binary(X, self.w_, add_bias = False).ravel()
        gradient = np.mean(X * ydiff[:, np.newaxis], axis = 0)
        
        # Regularization
        gradient = gradient.reshape(self.w_.shape)
        if self.penalty == "l1":
            gradient[1:] += -self.C * np.sign(self.w_[1:])
        elif self.penalty == "l2":
            gradient[1:] += -2 * self.C * self.w_[1:]
        elif self.penalty == "both":
            gradient[1:] += -self.C * (np.sign(self.w_[1:]) + 2 * self.w_[1:])
        
        # Return Gradient
        return gradient
    
    # Stochastic Gradient Calculation
    def _get_gradient_stochastic(self, X, y):

        # Grab Random Instance, Get Difference, Make Column Vector And Multiply Through
        idx = int(np.random.rand() * len(y))
        ydiff = y[idx] - self._predict_proba_binary(X[idx], self.w_, add_bias = False) 
        gradient = X[idx] * ydiff[:, np.newaxis]
        
        # Regularization
        gradient = gradient.reshape(self.w_.shape)
        if self.penalty == "l1":
            gradient[1:] += -self.C * np.sign(self.w_[1:])
        elif self.penalty == "l2":
            gradient[1:] += -2 * self.C * self.w_[1:]
        elif self.penalty == "both":
            gradient[1:] += -self.C * (np.sign(self.w_[1:]) + 2 * self.w_[1:])
        
        # Return Gradient
        return gradient
    
    # Newton's Method Gradient Calculation
    def _get_gradient_newton(self, X, y):

        # Get Sigmoid Value For All Classes, Calculate Hessian
        # Get Difference, Make Column Vector And Multiply Through
        g = self._predict_proba_binary(X, self.w_, add_bias = False).ravel()
        hessian = X.T @ np.diag(g * (1 - g)) @ X - 2 * self.C
        ydiff = y - g
        gradient = np.sum(X * ydiff[:, np.newaxis], axis = 0)
        
        # Regularization
        gradient = gradient.reshape(self.w_.shape)
        if self.penalty == "l1":
            gradient[1:] += -self.C * np.sign(self.w_[1:])
        elif self.penalty == "l2":
            gradient[1:] += -2 * self.C * self.w_[1:]
        elif self.penalty == "both":
            gradient[1:] += -self.C * (np.sign(self.w_[1:]) + 2 * self.w_[1:])
        
        # Return Gradient
        return pinv(hessian) @ gradient
    
    # ==============
    # Fit Functions
    # ==============

    def _fit_binary(self, X, y):

        # Add Bias Term
        Xb = self._add_bias(X)
        num_samples, num_features = Xb.shape
        
        # Initialize Weight Vectors To Zeros
        self.w_ = np.zeros((num_features, 1))
        
        # For Each Iteration (As Many As Max Iterations)
        for _ in range(self.iters):
            
            # Depending On Solver, Change Gradient Calculation
            if self.solver == "stochastic":
                # print("Stochastic Gradient Ascent Method")
                gradient = self._get_gradient_stochastic(Xb, y)
            elif self.solver == "newton":
                # print("Newton Method")
                gradient = self._get_gradient_newton(Xb, y)
            else:
                gradient = self._get_gradient(Xb, y)

            # Multiply By Learning Rate, Add B/C Maximizing
            self.w_ += gradient*self.eta

    def _fit_steepest(self, X, y):

         # Add Bias Term
        Xb = self._add_bias(X)
        num_samples, num_features = Xb.shape
        
        # Initialize Weight Vectors To Zeros
        self.w_ = np.zeros((num_features, 1))
        
        # For Each Iteration (As Many As Max Iterations)
        for _ in range(self.iters):

            # Get Gradient (Use Base Solver Since Steepest)
            # Minimization In Opposite Direction
            gradient = -self._get_gradient(Xb, y)
            
            # Do Line Search In Gradient Direction, Using Scipy Function (FIXME Line Iters Choice ? What Would Be Optimal)
            opts = {'maxiter' : self.line_iters}

            # Minimize Scalar Function (Line Search)
            res = minimize_scalar(self._objective_function_steepest,            # Objective Function To Optimize
                                  bounds = (0, self.eta * 10),                  # Bounds To Optimize Over
                                  args = (Xb, y, self.w_,gradient, self.C),     # Additional Argument For Objective Function
                                  method = 'bounded',                           # Bounded Optimization For Speed
                                  options = opts)                               # Set Maximum Iterations
            
            # Get Optimal Learning Rate, Set New Function Values, Subtract To Minimize
            eta = res.x
            self.w_ -= gradient * eta

    def _fit_bfgs(self, X, y):

        # Add Bias Term
        Xb = self._add_bias(X)
        num_samples, num_features = Xb.shape
        
        self.w_ = fmin_bfgs(self._objective_function_bfgs,                      # What To Optimize
                            np.zeros((num_features, 1)),                        # Starting Point
                            fprime = self._objective_gradient_bfgs,             # Gradient Function
                            args = (Xb, y, self.C),                             # Extra Arguments For Gradient, Objective Function
                            gtol = 1e-03,                                       # Stopping Criteria For Gradient
                            maxiter = self.iters,                               # Stopping Criteria Iterations
                            disp = False)
        
        self.w_ = self.w_.reshape((num_features, 1))
    
    # =================
    # Public Functions:
    # =================
    
    def predict_proba(self, X):

        # Initialize Probabilities
        probs = []

        # Get Probability For Each Classifier
        for w in self.classifiers_:
            probs.append(self._predict_proba_binary(X, w).reshape((len(X), 1)))

        # Make Into Single Matrix
        return np.hstack(probs)
    
    def predict(self, X):

        # Take Argmax Along Row
        return np.argmax(self.predict_proba(X), axis = 1)

    def fit(self, X, y):

        # Store Unique Classes
        num_samples, num_features = X.shape
        self.unique_ = np.sort(np.unique(y))
        num_unique_classes = len(self.unique_)
        self.classifiers_ = []

        # For Each Unique Value
        for i, yval in enumerate(self.unique_):

            # Create Binary Problem
            y_binary = np.array(y == yval).astype(int)
            
            # Train Binary Classifier For Class
            if self.solver == "steepest":
                # print("Steepest Ascent Method")
                self._fit_steepest(X, y_binary)
            elif self.solver == "quasi":
                # print("Quasi-Newton Method")
                self._fit_bfgs(X, y_binary)
            else:
                self._fit_binary(X, y_binary)

            # Add Trained Classifier To List
            self.classifiers_.append(self.w_)
            
        # Save All Weights Into One Matrix, Separate Column For Each Class
        self.w_ = np.hstack([w for w in self.classifiers_]).T

# **Testing Methods**

In [154]:
# Load Iris Dataset
iris = load_iris()
X = iris.data
y = iris.target

# Split Into Train Test Sets (80 / 20 Split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

# Scale Data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check Class Help
help(MultiClassLogisticRegression)

Help on class MultiClassLogisticRegression in module __main__:

class MultiClassLogisticRegression(builtins.object)
 |  MultiClassLogisticRegression(eta, iterations=20, C=0.0001, solver='quasi', penalty='l2', line_iters=10)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, eta, iterations=20, C=0.0001, solver='quasi', penalty='l2', line_iters=10)
 |      MultiClassLogisticRegression Constructor
 |      
 |      Note: Internally We Will Store The Weights As self.w_ To Keep With Sklearn Conventions
 |      
 |      Parameters:
 |      - eta: Learning Rate
 |      - iterations: Number Of Iterations (Default: 20)
 |      - C: Regularization Strength (Default: 0.0001)
 |      - solver: Optimization Algorithm: ('steepest', 'stochastic', 'newton', 'quasi') (Default: 'quasi')
 |      - penalty: Regularization Penalty Type: ('none', 'l1', 'l2', 'both') (Default: 'l2')
 |      - line_iters: Number Iterations For Line Search (If Applicable)
 |  
 |  __str__(self)
 |      Return str(self).
 |

In [155]:
%%time

# Instantiate Object
lr = MultiClassLogisticRegression(eta = 0.1, iterations = 100, solver = 'steepest', penalty = 'l2')

# Fit Model
lr.fit(X_train, y_train)

# Print Coefficients
print(lr)

# Predict
y_pred = lr.predict(X_test)

# Accuracy
print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))

MultiClass Logistic Regression Object With Coefficients:
[[-2.09332879 -0.93884016  1.89860074 -1.95702671 -1.79414871]
 [-1.00395114  0.34432948 -1.46008407  0.6501452  -1.13383002]
 [-2.99486496 -0.1293797  -0.16546227  2.08959157  3.3191224 ]]
Accuracy: 0.90
CPU times: total: 359 ms
Wall time: 460 ms


# **Other Tests (Not Trevor Code)**

In [42]:
%%time
lr = MultiClassLogisticRegression(eta=1.0,
                                  iterations=4,
                                  C=0.01,
                                  solver=BFGSBinaryLogisticRegression,
                                  penalty='l2'
                                 )
lr.fit(X_train,y_train)
print(lr)

yhat = lr.predict(X_test)
print('Accuracy of: ',accuracy_score(y_test,yhat))

MultiClass Logistic Regression Object with coefficients:
[[ 0.29042546  0.66260688  0.74538491 -1.13375801 -0.75909516]
 [-0.29042546 -0.66260688 -0.74538491  1.13375801  0.75909516]]
Accuracy of:  0.9666666666666667
CPU times: total: 0 ns
Wall time: 11.1 ms


In [53]:
%%time
lr = MultiClassLogisticRegression(eta=1.0,
                                  iterations=5,
                                  C=0.001,
                                  solver=HessianBinaryLogisticRegression,
                                  penalty='l1'
                                 )
lr.fit(X_train,y_train)
print(lr)

yhat = lr.predict(X_test)
print('Accuracy of: ',accuracy_score(y_test,yhat))

MultiClass Logistic Regression Object with coefficients:
[[ 15.79560094   1.9123695    1.14728782  -3.81520138  -7.45947451]
 [-15.79560094  -1.9123695   -1.14728782   3.81520138   7.45947451]]
Accuracy of:  0.9666666666666667
CPU times: total: 0 ns
Wall time: 3.5 ms


In [ ]:
%%time
# how do we compare now to sklearn?
from sklearn.linear_model import LogisticRegression

lr_sk = LogisticRegression(solver='lbfgs',n_jobs=1,
                           multi_class='ovr', C = 1/0.001, 
                           penalty='l2',
                          max_iter=50) # all params default
# note that sklearn is optimized for using the liblinear library with logistic regression
# ...and its faster than our implementation here

lr_sk.fit(X_train, y_train) # no need to add bias term, sklearn does it internally!!
print(lr_sk.coef_)
yhat = lr_sk.predict(X_test)
print('Accuracy of: ',accuracy_score(y_test,yhat))

In [ ]:
%%time
# actually, we aren't quite as good as the lib linear implementation
# how do we compare now to sklearn?
from sklearn.linear_model import LogisticRegression

lr_sk = LogisticRegression(solver='liblinear',n_jobs=1, 
                           multi_class='ovr', C = 1/0.001, 
                           penalty='l2',max_iter=100) 

lr_sk.fit(X_train,y_train) # no need to add bias term, sklearn does it internally!!
print(lr_sk.coef_)
yhat = lr_sk.predict(X_test)
print('Accuracy of: ',accuracy_score(y_test,yhat))

In [28]:
from sklearn.datasets import load_iris
import numpy as np
from sklearn.metrics import accuracy_score
from scipy.special import expit
from sklearn.model_selection import train_test_split

ds = load_iris()
X = ds.data
y = (ds.target>1).astype(int) # make problem binary
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.8, test_size=0.2)

In [30]:
%%time

#

blr = BinaryLogisticRegression(eta=0.1, iterations=50,C=0.001)

blr.fit(X_train,y_train)
print(blr)

yhat = blr.predict(X_test)
print('Accuracy of: ',accuracy_score(y_test,yhat))

#

lslr = LineSearchLogisticRegression(eta=1.0,
                                    iterations=6, 
                                    line_iters=8, 
                                    C=0.001)

lslr.fit(X_train,y_train)

yhat = lslr.predict(X_test)
print(lslr)
print('Accuracy of: ',accuracy_score(y_test,yhat))

#

slr = StochasticLogisticRegression(eta=0.01, iterations=800, C=0.001) # take a lot more steps!!

slr.fit(X_train,y_train)

yhat = slr.predict(X_test)
print(slr)
print('Accuracy of: ',accuracy_score(y_test,yhat))

#

hlr = HessianBinaryLogisticRegression(eta=1.0,
                                      iterations=4,
                                      C=0.001) # note that we need only a few iterations here

hlr.fit(X_train,y_train)
yhat = hlr.predict(X_test)
print(hlr)
print('Accuracy of: ',accuracy_score(y_test,yhat))

#

bfgslr = BFGSBinaryLogisticRegression(_,iterations=3,C=0.001) # note that we need only a few iterations here

bfgslr.fit(X_train,y_train)
yhat = bfgslr.predict(X_test)
print(bfgslr)
print('Accuracy of: ',accuracy_score(y_test,yhat))

Binary Logistic Regression Object with coefficients:
[[-0.19967076]
 [-0.47455757]
 [-0.49940795]
 [ 0.77090226]
 [ 0.53602837]]
Accuracy of:  0.9666666666666667
Binary Logistic Regression Object with coefficients:
[[-1.06296708]
 [-2.49138229]
 [-3.0503734 ]
 [ 4.23395947]
 [ 2.68939073]]
Accuracy of:  1.0
Binary Logistic Regression Object with coefficients:
[[-0.25950273]
 [-0.59364614]
 [-0.61723524]
 [ 1.04556247]
 [ 0.71349414]]
Accuracy of:  0.9333333333333333
Binary Logistic Regression Object with coefficients:
[[-12.02978114]
 [ -1.18083715]
 [ -0.13854738]
 [  1.98140576]
 [  6.15424339]]
Accuracy of:  0.9666666666666667
Binary Logistic Regression Object with coefficients:
[[-0.19732186]
 [-0.47166791]
 [-0.58692298]
 [ 0.86137527]
 [ 0.5429108 ]]
Accuracy of:  0.9666666666666667
CPU times: total: 0 ns
Wall time: 35.2 ms


C:\Users\trevo\AppData\Local\Temp\ipykernel_48532\906337921.py:17: RuntimeWarning: divide by zero encountered in log
  return -np.sum(ma.log(g[y==1]))-ma.sum(np.log(1-g[y==0])) + C*sum(wnew**2)
c:\Users\trevo\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\optimize\_optimize.py:2233: RuntimeWarning: invalid value encountered in subtract
  r = (xf - nfc) * (fx - ffulc)
c:\Users\trevo\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\optimize\_optimize.py:2234: RuntimeWarning: invalid value encountered in subtract
  q = (xf - fulc) * (fx - fnfc)
